# Request Type Analysis

Look at the request type values from 311.  Questions to consider:

  - Counts
  - Spatial (NCs) distribution
  - Time to complete
  - Time to complete by service provider
  - Spatial (service region) distribution?
  - ...

Steps in this notebook:

1.  Setup
2.  Create geodataframe/dataframe from cleaned data and [census](https://data.lacity.org/Community-Economic-Development/Census-Data-by-Neighborhood-Council/nwj3-ufba)
3.  Examine the data
4.  Compute the measure
5.  Show measure as choropleth
6.  So what (next steps)

# 1 - Setup

In [1]:
%run start.py
from utils import read_new311_shape, dt_to_object

2021-11-29 19:39:10 Configured OSMnx 1.1.1
2021-11-29 19:39:10 HTTP response caching is on


# 2 - Get Data Files

Using two data files:

  - The Certified Neighborhoods cleaned up in NC-service-regions.ipynb
  - [Census](https://data.lacity.org/Community-Economic-Development/Census-Data-by-Neighborhood-Council/nwj3-ufba) data from data.lacity.org
  
**Note** - This is 2010 Census data.  Probably need to look for 2020.

In [2]:
%%time
graffiti_gdf = read_new311_shape('../data/311/graffiti.zip/')

CPU times: user 28.7 s, sys: 1.21 s, total: 29.9 s
Wall time: 30.3 s


In [11]:
graffiti_gdf.columns

Index(['SRNumber', 'created_dt', 'updated_dt', 'owner', 'request_type',
       'service_dt', 'closed_dt', 'address', 'street', 'zip_code', 'latitude',
       'longitude', 'location', 'APC', 'cd', 'cd_member', 'nc', 'nc_name',
       'precinct', 'days_to_service', 'days_to_close', 'days_to_update',
       'service_region', 'region_id', 'marker_color', 'popup_message',
       'geometry'],
      dtype='object')

In [25]:
graffiti_gdf['owner'].value_counts()

OCB    315577
Name: owner, dtype: int64

In [3]:
neighborhoods_gdf = gpd.read_file('../data/neighborhoods/Neighborhood_Councils_(Certified)_cleaned.zip/')

neighborhoods_gdf.rename(columns={'NAME': 'name',
                        'NC_ID': 'nc_id',
                        'SERVICE_RE': 'service_region'},
              inplace=True);

In [4]:
neighborhoods_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   OBJECTID        99 non-null     int64   
 1   name            99 non-null     object  
 2   WADDRESS        99 non-null     object  
 3   DWEBSITE        99 non-null     object  
 4   DEMAIL          98 non-null     object  
 5   DPHONE          99 non-null     object  
 6   nc_id           99 non-null     int64   
 7   CERTIFIED       99 non-null     object  
 8   TOOLTIP         99 non-null     object  
 9   NLA_URL         99 non-null     object  
 10  service_region  99 non-null     object  
 11  region_id       99 non-null     object  
 12  color_code      99 non-null     object  
 13  geometry        99 non-null     geometry
dtypes: geometry(1), int64(2), object(11)
memory usage: 11.0+ KB


# 3 - Quick Looksie

Well, there's a discrepancy here.  The census data has 97 NC's and the certified dataset has 99 (I think the right number is 99).

Not going to agonize over this at this stage but want to understand things.  Adjusting for what matches as this stage should be good enough for now.

In [15]:
graffiti_counts = graffiti_gdf['nc'].value_counts().to_frame().reset_index().rename(columns={'index': 'nc_id', 'nc': 'count'})

In [16]:
graffiti_counts

,nc_id,count
0,78,26836
1,50,18197
2,52,15584
3,125,13082
4,86,10799
...,...,...
94,63,238
95,64,221
96,126,217
97,114,140


In [17]:
graffiti_merged = pd.merge(neighborhoods_gdf, graffiti_counts, how="left", on=["nc_id"])

In [18]:
graffiti_merged

,OBJECTID,name,WADDRESS,DWEBSITE,DEMAIL,DPHONE,nc_id,CERTIFIED,TOOLTIP,NLA_URL,service_region,region_id,color_code,geometry,count
0,1,ARLETA NC,http://www.arletanc.org/,http://empowerla.org/ANC,ANC@EmpowerLA.org,213-978-1551,6,2002-10-22,ARLETA NC,navigatela/reports/nc_reports.cfm?id=6,REGION 1 - NORTH EAST VALLEY,1,#00BFFF,"POLYGON ((-118.45006 34.24992, -118.45057 34.2...",1896
1,2,ARROYO SECO NC,http://www.asnc.us/,http://empowerla.org/ASNC,ASNC@EmpowerLA.org,213-978-1551,42,2002-10-02,ARROYO SECO NC,navigatela/reports/nc_reports.cfm?id=42,REGION 8 - NORTH EAST LA,8,#FF8C00,"POLYGON ((-118.22326 34.10393, -118.22368 34.1...",869
2,3,ATWATER VILLAGE NC,http://www.atwatervillage.org/,http://empowerla.org/AVNC,AVNC@EmpowerLA.org,213-978-1551,37,2003-02-11,ATWATER VILLAGE NC,navigatela/reports/nc_reports.cfm?id=37,REGION 7 - EAST,7,#87CEEB,"POLYGON ((-118.27577 34.15377, -118.26185 34.1...",929
3,4,BEL AIR-BEVERLY CREST NC,http://babcnc.org/,http://empowerla.org/BABCNC,BABCNC@EmpowerLA.org,213-978-1551,64,2002-10-08,BEL AIR-BEVERLY CREST NC,navigatela/reports/nc_reports.cfm?id=64,REGION 11 - WEST LA,11,#7FFFD4,"POLYGON ((-118.47487 34.12635, -118.47412 34.1...",221
4,5,BOYLE HEIGHTS NC,http://bhnc.net/,http://empowerla.org/BHNC,BHNC@EmpowerLA.org,213-978-1551,50,2002-05-21,BOYLE HEIGHTS NC,navigatela/reports/nc_reports.cfm?id=50,REGION 8 - NORTH EAST LA,8,#FF8C00,"POLYGON ((-118.21441 34.06064, -118.21305 34.0...",18197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,95,WILMINGTON NC,http://www.wilmingtonneighborhoodcouncil.com/,http://empowerla.org/WILMINGTON,WilmingtonNC@EmpowerLA.org,213-978-1551,93,2001-12-11,WILMINGTON NC,navigatela/reports/nc_reports.cfm?id=93,REGION 12 - HARBOR,12,#DB7093,"POLYGON ((-118.28303 33.79825, -118.27531 33.7...",5645
95,96,WILSHIRE CENTER - KOREATOWN NC,http://wcknc.org/,http://empowerla.org/WCKNC,WCKNC@EmpowerLA.org,213-978-1551,55,2003-08-05,WILSHIRE CENTER - KOREATOWN NC,navigatela/reports/nc_reports.cfm?id=55,REGION 6 - CENTRAL 2,6,#CD853F,"POLYGON ((-118.29172 34.08018, -118.29167 34.0...",7887
96,97,WINNETKA NC,http://www.winnetkanc.com/,http://empowerla.org/WINNETKA,WinnetkaNC@EmpowerLA.org,213-978-1551,14,2003-07-22,WINNETKA NC,navigatela/reports/nc_reports.cfm?id=14,REGION 3 - SOUTH WEST VALLEY,3,#EE82EE,"POLYGON ((-118.58856 34.23547, -118.58347 34.2...",1025
97,98,WOODLAND HILLS-WARNER CENTER NC,http://www.whcouncil.org/index.php,http://empowerla.org/WHWCNC,WHWCNC@EmpowerLA.org,213-978-1551,16,2002-03-06,WOODLAND HILLS-WARNER CENTER NC,navigatela/reports/nc_reports.cfm?id=16,REGION 3 - SOUTH WEST VALLEY,3,#EE82EE,"POLYGON ((-118.56221 34.17312, -118.56220 34.1...",2050


# 4 - Compute the Measure

Computation is simple.  Use the geometry of the NC to compute area in miles squared.

For the density I'm simply using total population.  I suspect it would be interesting to examine some of the other ethnic measures?  Maybe a nice pull down to select?  Ah... for another day.

In [ ]:
from pyproj import Geod

geod = Geod(ellps="WGS84")

def square_miles(geo):
    square_meters = abs(geod.geometry_area_perimeter(geo)[0])
    return (square_meters * 10.764) / 27878000

In [ ]:
neighborhood_merged['sq_miles'] = neighborhood_merged.apply(lambda row: square_miles(row.geometry), axis=1)

In [ ]:
neighborhood_merged['density'] = neighborhood_merged.apply(lambda row: row['Total Population'] / row['sq_miles'], axis=1)

Remember I like to look at one of the values.

In [ ]:
neighborhood_merged.iloc[27]

Some sanity checking on the data before we generate the display.

In the real world we'll have to do some more work on this data!

In [ ]:
neighborhood_merged.density.max()

In [ ]:
neighborhood_merged.density.min()

In [ ]:
len(neighborhood_merged)

# 5 - Display the Choropleth

In [19]:
imagery = basemap_to_tiles(basemaps.Esri.WorldImagery)
imagery.base = True
osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
osm.base = True


map_display = Map(center=(34.05, -118.25), zoom=12,
                  layers=[imagery, osm],
                  layout=Layout(height="900px"),
                  scroll_wheel_zoom=True)

#map_display.add_control(LayersControl())
#map_display += nc_layer
map_display

Map(center=[34.05, -118.25], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

refer to : https://www.youtube.com/watch?v=wjzAy_yLrdA

In [22]:
from ipyleaflet import Choropleth, Map
from branca.colormap import linear
a_geojson = json.loads(graffiti_merged.to_json())

graffiti_density = dict(zip(graffiti_merged['name'].tolist(), graffiti_merged['count'].tolist()))
for i in a_geojson['features']:
    i['id'] = i['properties']['name']

layer = Choropleth(
                    geo_data=a_geojson,
                    choro_data=graffiti_density,
                    colormap=linear.YlOrRd_09, #linear.Blues_05,
                    style={'fillOpacity': 1.0, "color":"black"},)
                    #key_on="name")

map_display.add_layer(layer)

I need to revisit a tooltip type popup.  For now this will work.

In [24]:
geo_json = GeoJSON(
    data=a_geojson,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.6, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    name='NCs'
)

html = HTML('''Hover over a district''')
html.layout.margin = '0px 20px 20px 20 px'
control = WidgetControl(widget=html, position='bottomright')

def update_html(feature, **kwargs):
    html.value = '''<h3><b>NC: {}</b></h3>
                    <h4>Count: {}'''.format(feature['properties']['name'],
                                                           feature['properties']['count'])
    
map_display.add_control(control)  # does += work for this?

layer.on_hover(update_html)

# 6 - So What?

I say this tounge in cheeck.  Things to think about:

  1. Should we examine measures besides total population?
  2. Does it make sense to extend the 311 data as we did with the service regions?
  3. Do we just use this to select an NC then query 311 (or ...)?
  
